In [ ]:
import sys
sys.path.extend([ '../', '../../', '../../../'])
import torch
import dataloader as dl
from args import Args
# from model import Het_ConEn, Het_NetEn, EdgePredictor, Het_classify
from model import MAG_ConEn, MAG_NetEn, EdgePredictor, MAG_classify
from train import train_smote

# Set device to GPU if available, else use CPU
args = Args()
args.aminer_train()
torch.cuda.empty_cache()

Current device: NVIDIA A100 80GB PCIe


In [2]:
data = torch.load('../../../data/aminer/am_data.pt')
print(data)

HeteroData(
  a={
    num_nodes=20171,
    y=[20171],
  },
  p={ num_nodes=13250 },
  v={ num_nodes=18 },
  p_title_embed={ x=[13250, 128] },
  p_abstract_embed={ x=[13250, 128] },
  p_net_embed={ x=[13250, 128] },
  p_a_net_embed={ x=[13250, 128] },
  p_p_net_embed={ x=[13250, 128] },
  p_v_net_embed={ x=[13250, 128] },
  a_net_embed={ x=[20171, 128] },
  a_text_embed={ x=[20171, 128] },
  v_net_embed={ x=[18, 128] },
  v_text_embed={ x=[18, 128] },
  (a, walk, a)={ edge_index=[2, 99480] },
  (a, walk, p)={ edge_index=[2, 86444] },
  (a, walk, v)={ edge_index=[2, 31716] },
  (p, walk, a)={ edge_index=[2, 65534] },
  (p, walk, p)={ edge_index=[2, 66174] },
  (p, walk, v)={ edge_index=[2, 20838] },
  (v, walk, a)={ edge_index=[2, 90] },
  (v, walk, p)={ edge_index=[2, 90] },
  (v, walk, v)={ edge_index=[2, 27] }
)


In [3]:
print(data['a','walk','p'].edge_index)
device = args.device
# data = data.to(device)

# Send all x tensors to the device
data['p_title_embed']['x'] = data['p_title_embed']['x'].to(device)
data['p_abstract_embed']['x'] = data['p_abstract_embed']['x'].to(device)
data['p_net_embed']['x'] = data['p_net_embed']['x'].to(device)
data['p_a_net_embed']['x'] = data['p_a_net_embed']['x'].to(device)
data['p_p_net_embed']['x'] = data['p_p_net_embed']['x'].to(device)
data['p_v_net_embed']['x'] = data['p_v_net_embed']['x'].to(device)
data['a_net_embed']['x'] = data['a_net_embed']['x'].to(device)
data['a_text_embed']['x'] = data['a_text_embed']['x'].to(device)
data['v_net_embed']['x'] = data['v_net_embed']['x'].to(device)
data['v_text_embed']['x'] = data['v_text_embed']['x'].to(device)

# Send all y tensors to the device
data['a']['y'] = data['a']['y'].to(device)

# Send all edge_index tensors to the device
data['a', 'walk', 'a']['edge_index'] = data['a', 'walk', 'a']['edge_index'].to(device)
data['a', 'walk', 'p']['edge_index'] = data['a', 'walk', 'p']['edge_index'].to(device)
data['a', 'walk', 'v']['edge_index'] = data['a', 'walk', 'v']['edge_index'].to(device)
data['p', 'walk', 'a']['edge_index'] = data['p', 'walk', 'a']['edge_index'].to(device)
data['p', 'walk', 'p']['edge_index'] = data['p', 'walk', 'p']['edge_index'].to(device)
data['p', 'walk', 'v']['edge_index'] = data['p', 'walk', 'v']['edge_index'].to(device)
data['v', 'walk', 'a']['edge_index'] = data['v', 'walk', 'a']['edge_index'].to(device)
data['v', 'walk', 'p']['edge_index'] = data['v', 'walk', 'p']['edge_index'].to(device)
data['v', 'walk', 'v']['edge_index'] = data['v', 'walk', 'v']['edge_index'].to(device)

edge_indices = [ data['a', 'walk', 'a'].edge_index, data['a', 'walk', 'p'].edge_index, data['a', 'walk', 'v'].edge_index ]

tensor([[    0,     0,     0,  ..., 20170, 20170, 20170],
        [10523, 10442, 10329,  ...,  4378, 12456,  6612]])


In [4]:
c_train_num = dl.train_num(data['a'].y, args.im_class_num, args.class_samp_num[0], args.im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['a'].y, c_train_num, args.seed[1], args)
print("train_idx: ", train_idx, len(train_idx))
print("val_idx: ", val_idx, len(val_idx))
print("test_idx: ", test_idx, len(test_idx))
# print(c_num_mat)

[50, 30, 25, 20] 125
0 335
1 202
2 299
3 289
train_idx:  [19186, 15790, 10435, 10918, 14011, 11422, 17533, 9706, 8711, 2379, 10819, 9056, 13810, 9029, 7125, 9469, 9740, 17900, 6050, 1731, 8151, 13157, 18289, 14814, 19337, 11409, 2931, 16505, 17095, 1730, 4663, 11413, 13096, 18983, 17386, 2073, 2985, 16274, 7953, 3376, 17692, 7212, 8890, 7902, 3968, 15076, 607, 6772, 19647, 13844, 15393, 6076, 4172, 6837, 17201, 151, 6616, 4475, 17635, 1510, 14304, 15542, 3174, 1943, 15168, 7909, 16357, 19283, 11517, 8657, 13074, 18093, 296, 7808, 17783, 2933, 2420, 13942, 4322, 17669, 16931, 6433, 14221, 14559, 10021, 3437, 18608, 8410, 17438, 17356, 9688, 6716, 9860, 2876, 11500, 16477, 18798, 11434, 10797, 16685, 8507, 16623, 13934, 1260, 11651, 17939, 16232, 6470, 16278, 8065, 11823, 14914, 2741, 14143, 384, 3413, 121, 9390, 3355, 9247, 1119, 10349, 10955, 15417, 7602] 125
val_idx:  [13072, 16485, 19168, 15984, 347, 17541, 19108, 18889, 9180, 917, 7986, 19234, 10693, 19984, 5782, 13137, 19079, 68, 2

In [5]:
### MAGNN

encoder1 = MAG_ConEn(args.embed_dim, args.dropout)
encoder2 = MAG_NetEn(args.embed_dim, args.dropout)
classifier = MAG_classify(args.embed_dim, args.nclass, args.dropout)
decoder_a = EdgePredictor(args.embed_dim)
decoder_p = EdgePredictor(args.embed_dim)
decoder_v = EdgePredictor(args.embed_dim)
# decoder_a = torch.load('pretrained/decoder_a.pth')
# decoder_p = torch.load('pretrained/decoder_p.pth')
# decoder_v = torch.load('pretrained/decoder_v.pth')
decoder_list = [decoder_a, decoder_p, decoder_v]

encoder1.to(device)
encoder2.to(device)
classifier.to(device)
for decoder in decoder_list:
    decoder.to(device)

In [6]:
### HETGNN

# encoder1 = Het_ConEn(args.embed_dim, args.dropout)
# encoder2 = Het_NetEn(args.embed_dim, args.dropout)
# classifier = Het_classify(args.embed_dim, args.nclass, args.dropout)
# decoder_a = EdgePredictor(args.embed_dim)
# decoder_p = EdgePredictor(args.embed_dim)
# decoder_v = EdgePredictor(args.embed_dim)
# # decoder_a = torch.load('pretrained/decoder_a.pth')
# # decoder_p = torch.load('pretrained/decoder_p.pth')
# # decoder_v = torch.load('pretrained/decoder_v.pth')
# decoder_list = [decoder_a, decoder_p, decoder_v]

# encoder1.to(device)
# encoder2.to(device)
# classifier.to(device)
# for decoder in decoder_list:
#     decoder.to(device)

## Training Part

In [7]:
torch.cuda.empty_cache()
train_smote(data, edge_indices, encoder1, encoder2, classifier, decoder_list, 
            train_idx, val_idx, test_idx, args, os_mode = 'gsm', train_mode = 'recon')

[tensor(0.0001, device='cuda:0', grad_fn=<DivBackward0>)]
[tensor(0.0001, device='cuda:0', grad_fn=<DivBackward0>), tensor(8.7271e-05, device='cuda:0', grad_fn=<DivBackward0>)]
[tensor(0.0001, device='cuda:0', grad_fn=<DivBackward0>), tensor(8.7271e-05, device='cuda:0', grad_fn=<DivBackward0>), tensor(2.0592e-05, device='cuda:0', grad_fn=<DivBackward0>)]
Epoch [1/200], Loss: 0.0705, Accuracy: 0.2500, Edge Accuracy: [0.9997554988434233, 0.9996765616245255, 0.9126468692677606]
class 0: 0.1, Class 0: AUC-ROC- 0.5268, F1 Score- 0.4000; class 1: 0.1, Class 1: AUC-ROC- 0.5011, F1 Score- 0.4000; class 2: 0.1, Class 2: AUC-ROC- 0.5327, F1 Score- 0.4000; class 3: 0.1, Class 3: AUC-ROC- 0.5033, F1 Score- 0.4000; Macro-Average AUC-ROC: 0.5160; Macro-Average F1 Score: 0.4000
Validation Loss: 0.0000, Validation Accuracy: 0.2500, Validation Edge Accuracy: []
class 0: 0.1, Class 0: AUC-ROC- 0.1467, F1 Score- 0.4000; class 1: 0.1, Class 1: AUC-ROC- 0.3268, F1 Score- 0.4000; class 2: 0.1, Class 2: AUC-

([0.25,
  0.25,
  0.25,
  0.25,
  0.25,
  0.25,
  0.25,
  0.25,
  0.25,
  0.25,
  0.25,
  0.25,
  0.25,
  0.25,
  0.25,
  0.25,
  0.25,
  0.25,
  0.25,
  0.25,
  0.25],
 [0.5190444444444444,
  0.5160814814814815,
  0.5156888888888889,
  0.5146666666666667,
  0.5129925925925927,
  0.5079777777777778,
  0.5014444444444445,
  0.4978444444444444,
  0.4962,
  0.49768888888888885,
  0.4984222222222223,
  0.5008740740740741,
  0.509437037037037,
  0.5335777777777777,
  0.5717111111111111,
  0.6044444444444445,
  0.6267037037037038,
  0.6513185185185186,
  0.6687037037037038,
  0.6766000000000001,
  0.6781185185185186],
 [0.4,
  0.4,
  0.4,
  0.4,
  0.4,
  0.4,
  0.4,
  0.4,
  0.4,
  0.4,
  0.4,
  0.4,
  0.4,
  0.4,
  0.4,
  0.4,
  0.4,
  0.4,
  0.4,
  0.4,
  0.4],
 [0.4454518518518519,
  0.32482962962962963,
  0.6855407407407407,
  0.9493925925925926],
 [0.4, 0.4, 0.4, 0.4])

In [ ]:
# torch.save(decoder_list[0], '../pretrained_magnn/decoder_a.pth')
# torch.save(decoder_list[1], '../pretrained_magnn/decoder_p.pth')
# torch.save(decoder_list[2], '../pretrained_magnn/decoder_v.pth')
# torch.save(encoder1, '../pretrained_magnn/encoder1.pth')
# torch.save(encoder2, '../pretrained_magnn/encoder2.pth')